In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging

# Configure logging for Fonduer
logging.basicConfig(stream=sys.stdout, format='[%(levelname)s] %(name)s:%(lineno)s - %(message)s')
logger = logging.getLogger('fonduer')
logger.setLevel(logging.INFO)

PARALLEL = 16 # assuming a quad-core machine
ATTRIBUTE = "circular_connectors"
conn_string = 'postgresql://localhost:5432/' + ATTRIBUTE

In [2]:
# If you've run this before, set FIRST_TIME to False to save time
FIRST_TIME = False

In [3]:
from fonduer import Meta

session = Meta.init(conn_string).Session()

[INFO] fonduer.meta:86 - Connecting user:None to localhost:5432/circular_connectors
[INFO] fonduer.meta:110 - Initializing the storage schema


In [4]:
from hack.utils import parse_dataset

dirname = "."

docs, train_docs, dev_docs, test_docs = parse_dataset(
    session, dirname, first_time=FIRST_TIME, parallel=PARALLEL, max_docs=100
)
logger.info(f"# of train Documents: {len(train_docs)}")
logger.info(f"# of dev Documents: {len(dev_docs)}")
logger.info(f"# of test Documents: {len(test_docs)}")

[INFO] fonduer:8 - # of train Documents: 100
[INFO] fonduer:9 - # of dev Documents: 100
[INFO] fonduer:10 - # of test Documents: 98


In [5]:
from fonduer.parser.models import Document, Section, Paragraph, Sentence, Figure

logger.info(f"Documents: {session.query(Document).count()}")
logger.info(f"Sections: {session.query(Section).count()}")
logger.info(f"Paragraphs: {session.query(Paragraph).count()}")
logger.info(f"Sentences: {session.query(Sentence).count()}")
logger.info(f"Figures: {session.query(Figure).count()}")

[INFO] fonduer:3 - Documents: 298
[INFO] fonduer:4 - Sections: 298
[INFO] fonduer:5 - Paragraphs: 330839
[INFO] fonduer:6 - Sentences: 341046
[INFO] fonduer:7 - Figures: 21269


In [6]:
from fonduer.candidates.models import mention_subclass

Thumbnails = mention_subclass("Thumbnails")

In [7]:
from fonduer.candidates import MentionFigures

thumbnails_img = MentionFigures()

In [8]:
from fonduer.candidates.matchers import _Matcher

In [9]:
from PIL import Image

class HasFigures(_Matcher):   
    def _f(self, m):
        file_path = ""
        for prefix in ["data/train/html/", "data/dev/html/", "data/test/html/"]:
            if os.path.exists(prefix + m.figure.url):
                file_path = prefix + m.figure.url
        if file_path == "":
            return False
        img = Image.open(file_path)
        width, height = img.size
        min_value = min(width, height)
        return min_value > 50

In [10]:
from fonduer.candidates import MentionExtractor
from fonduer.candidates.matchers import DoNothingMatcher

mention_extractor = MentionExtractor(
    session, [Thumbnails], [thumbnails_img], [HasFigures()], parallelism=PARALLEL
)

from fonduer.candidates.models import Mention

if FIRST_TIME:
    mention_extractor.apply(docs)

logger.info("Total Mentions: {}".format(session.query(Mention).count()))

[INFO] fonduer:13 - Total Mentions: 8917


In [11]:
from fonduer.candidates.models import candidate_subclass

ThumbnailLabel = candidate_subclass("ThumbnailLabel", [Thumbnails])

In [12]:
from fonduer.candidates import CandidateExtractor

candidate_extractor = CandidateExtractor(
    session, [ThumbnailLabel], throttlers=[None], parallelism=PARALLEL
)

if FIRST_TIME or True:
    candidate_extractor.apply(train_docs, split=0)
    candidate_extractor.apply(dev_docs, split=1)
    candidate_extractor.apply(test_docs, split=2)

[INFO] fonduer.candidates.candidates:125 - Clearing table thumbnail_label (split 0)
[INFO] fonduer.utils.udf:57 - Running UDF...



[INFO] fonduer.candidates.candidates:125 - Clearing table thumbnail_label (split 1)
[INFO] fonduer.utils.udf:57 - Running UDF...



[INFO] fonduer.candidates.candidates:125 - Clearing table thumbnail_label (split 2)
[INFO] fonduer.utils.udf:57 - Running UDF...


In [13]:
train_cands = candidate_extractor.get_candidates(split=0)
dev_cands = candidate_extractor.get_candidates(split=1)
test_cands = candidate_extractor.get_candidates(split=2)

In [14]:
logger.info("Total train candidate:\t{}".format(len(train_cands[0])))
logger.info("Total dev candidate:\t{}".format(len(dev_cands[0])))
logger.info("Total test candidate:\t{}".format(len(test_cands[0])))

[INFO] fonduer:1 - Total train candidate:	7256
[INFO] fonduer:2 - Total dev candidate:	453
[INFO] fonduer:3 - Total test candidate:	1208


In [15]:
fin = open("data/ground_truth.txt", "r")
gt = set()
for line in fin:
    gt.add("::".join(line.lower().split()))
fin.close()
# gt

In [16]:
TRUE=1
FALSE=2
ABSTAIN=0

In [17]:
def LF_gt_label(c):
    doc_file_id = f"{c[0].context.figure.document.name.lower()}.pdf::{os.path.basename(c[0].context.figure.url.lower())}"
#     print(doc_file_id)
    return TRUE if doc_file_id in gt else FALSE

In [18]:
ans = {0:0, 1:0, 2:0}

gt_dev_pb = []
gt_dev = []
gt_test = []

In [19]:
for cand in dev_cands[0]:
    if LF_gt_label(cand) == 1:
        ans[1] += 1
        gt_dev_pb.append([1., 0.])
        gt_dev.append(1.)
    else:
        ans[2] += 1
        gt_dev_pb.append([0., 1.])
        gt_dev.append(2.)

In [20]:
ans

{0: 0, 1: 69, 2: 384}

In [21]:
ans = {0:0, 1:0, 2:0}

In [22]:
for cand in test_cands[0]:
    gt_test.append(LF_gt_label(cand))
    ans[gt_test[-1]] += 1

In [23]:
ans

{0: 0, 1: 160, 2: 1048}

In [24]:
batch_size = 64
input_size = 224

In [25]:
from disc_model.torchnet import *
from utils import *

In [27]:
# train_loader = torch.utils.data.DataLoader(
#     ImageList(
#         data=all_cands,
#         label=torch.Tensor(all_label),
# #         label=torch.Tensor(gt_dev_pb),
#         transform=transform(input_size),
#         prefix="data/dev/html/",
#     ),
#     batch_size=batch_size,
#     shuffle = True,
# #     sampler = sampler
# )

In [28]:
train_loader = torch.utils.data.DataLoader(
    ImageList(
        data=dev_cands[0],
#         label=torch.Tensor(gt_dev),
        label=torch.Tensor(gt_dev_pb),
        transform=transform(input_size),
        prefix="data/dev/html/",
    ),
    batch_size=batch_size,
    shuffle = False,
#     sampler = sampler
)

In [29]:
dev_loader = torch.utils.data.DataLoader(
    ImageList(
        data=dev_cands[0],
#         label=torch.Tensor(gt_dev),
        label=gt_dev,
        transform=transform(input_size),
        prefix="data/dev/html/",
    ),
    batch_size=batch_size,
    shuffle=False,
)

In [30]:
test_loader = torch.utils.data.DataLoader(
    ImageList(
        data=test_cands[0],
        label=gt_test,
        transform=transform(input_size),
        prefix="data/test/html/",
    ),
    batch_size=100,
    shuffle=False,
)

In [31]:
from metal import EndModel

In [32]:
em_config = {
    # GENERAL
    "seed": None,
    "verbose": True,
    "show_plots": True,
    # Network
    # The first value is the output dim of the input module (or the sum of
    # the output dims of all the input modules if multitask=True and
    # multiple input modules are provided). The last value is the
    # output dim of the head layer (i.e., the cardinality of the
    # classification task). The remaining values are the output dims of
    # middle layers (if any). The number of middle layers will be inferred
    # from this list.
    #     "layer_out_dims": [10, 2],
    # Input layer configs
    "input_layer_config": {
        "input_relu": False,
        "input_batchnorm": False,
        "input_dropout": 0.0,
    },
    # Middle layer configs
    "middle_layer_config": {
        "middle_relu": False,
        "middle_batchnorm": False,
        "middle_dropout": 0.0,
    },
    # Can optionally skip the head layer completely, for e.g. running baseline
    # models...
    "skip_head": True,
    # GPU
    "use_cuda": True,
    # MODEL CLASS
    "resnet18"
    # DATA CONFIG
    "src": "gm",
    # TRAINING
    "train_config": {
        # Display
        "print_every": 1,  # Print after this many epochs
        "disable_prog_bar": False,  # Disable progress bar each epoch
        # Dataloader
        "data_loader_config": {"batch_size": 32, "num_workers": 8, "sampler": None},
        # Loss weights
        "loss_weights": [0.5, 0.5],
        # Train Loop
        "n_epochs": 20,
        # 'grad_clip': 0.0,
        "l2": 0.0,
        # "lr": 0.01,
        "validation_metric": "accuracy",
        "validation_freq": 1,
        # Evaluate dev for during training every this many epochs
        # Optimizer
        "optimizer_config": {
            "optimizer": "adam",
            "optimizer_common": {"lr": 0.01},
            # Optimizer - SGD
            "sgd_config": {"momentum": 0.9},
            # Optimizer - Adam
            "adam_config": {"betas": (0.9, 0.999)},
        },
        # Scheduler
        "scheduler_config": {
            "scheduler": "reduce_on_plateau",
            # ['constant', 'exponential', 'reduce_on_plateu']
            # Freeze learning rate initially this many epochs
            "lr_freeze": 0,
            # Scheduler - exponential
            "exponential_config": {"gamma": 0.9},  # decay rate
            # Scheduler - reduce_on_plateau
            "plateau_config": {
                "factor": 0.5,
                "patience": 1,
                "threshold": 0.0001,
                "min_lr": 1e-5,
            },
        },
        # Checkpointer
        "checkpoint": True,
        "checkpoint_config": {
            "checkpoint_min": -1,
            # The initial best score to beat to merit checkpointing
            "checkpoint_runway": 0,
            # Don't start taking checkpoints until after this many epochs
        },
    },
}

In [35]:
from metal.tuners import RandomSearchTuner
# from metal.contrib.logging.tensorboard import TensorBoardWriter

log_config = {"log_dir": "./run_logs", "run_name": "image"}

tuner_config = {"max_search": 3}
search_space = {
    "l2": [0.001, 0.0001, 0.00001],  # linear range
    "lr": {"range": [0.0001, 0.1], "scale": "log"},  # log range
}


train_config = em_config["train_config"]


# Defining network parameters
num_classes = 2
fc_size = 2
hidden_size = 2
pretrained = True

# Set CUDA device
torch.cuda.set_device(1)
# os.environ['CUDA_VISIBLE_DEVICES']='0'

# Initializing input module
input_module = get_cnn("resnet18", pretrained=pretrained, num_classes=num_classes)


# Initializing model object
init_args = [[num_classes]]
init_kwargs = {"input_module": input_module}
init_kwargs.update(em_config)

# init_kwargs.update(em_config)
# max_search = tuner_config['max_search']
# metric = train_config['validation_metric']

# Training model as a single pass
# if args.single_pass:
# end_model = EndModel(
#    [hidden_size, fc_size, num_classes],
#    input_module=input_module,use_cuda='True'
#     **em_config
# )
# end_model.train_model(
#    train_data=train_loader,
#    dev_data=dev_loader,
#     **train_config
# )

# Searching model
# else:
searcher = RandomSearchTuner(EndModel, **log_config)

end_model = searcher.search(
    search_space,
    dev_loader,
    train_args=[train_loader],
    init_args=init_args,
    init_kwargs=init_kwargs,
    train_kwargs=train_config,
    max_search=tuner_config["max_search"],
)

# Evaluating model
scores = end_model.score(
    test_loader, metric=["accuracy", "precision", "recall", "f1", "roc-auc"]
)

labels, _, probs = end_model._get_predictions(test_loader, return_probs=True)

  0%|          | 0/8 [00:00<?, ?it/s]

Using class weight vector [0.5, 0.5]...

Network architecture:
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

  0%|          | 0/8 [00:00<?, ?it/s]

Saving model at iteration 0 with best score 0.848
[E:0]	Train Loss: 0.816	Dev score: 0.848


  0%|          | 0/8 [00:00<?, ?it/s]

...

[E:17]	Train Loss: 0.009	Dev score: 0.993


  0%|          | 0/8 [00:00<?, ?it/s]

[E:18]	Train Loss: 0.009	Dev score: 0.993


100%|██████████| 8/8 [00:04<00:00,  1.75it/s, avg_loss=0.0976]  


[E:19]	Train Loss: 0.009	Dev score: 0.993
Restoring best model from iteration 13 with score 0.993
Finished Training
Accuracy: 0.993
        y=1    y=2   
 l=1    69      3    
 l=2     0     381   
[SUMMARY]
Best model: [1]
Best config: {'l2': 0.0001, 'lr': 0.00019105920782773857}
Best score: 1.0
Accuracy: 0.904
Precision: 0.588
Recall: 0.919
F1: 0.717
Roc-auc: 0.950
        y=1    y=2   
 l=1    147    103   
 l=2    13     945   


In [ ]:
labels, _, probs = end_model._get_predictions(test_loader, return_probs=True)

```
============================================================
[SUMMARY]
Best model: [0]
Best config: {'l2': 0.0001, 'lr': 0.0010025532524850966}
Best score: 0.9955849889624724
============================================================
Accuracy: 0.926
Precision: 0.675
Recall: 0.856
F1: 0.755
Roc-auc: 0.964
        y=1    y=2   
 l=1    137    66    
 l=2    23     982   
```

In [ ]:
gt_test

In [ ]:
pred_dict = {}

for c, y in zip(test_cands[0], probs[:,0]):
    doc_file_id = f"{c[0].context.figure.document.name.lower()}.pdf::{os.path.basename(c[0].context.figure.url.lower())}"
    pred_dict[doc_file_id] = y

In [ ]:
all_test_fig_id = set()

for doc in test_docs:
    for fig in doc.figures:
        doc_file_id = f"{doc.name.lower()}.pdf::{os.path.basename(fig.url.lower())}"
        all_test_fig_id.add(doc_file_id)

In [ ]:
b = 0.7

tp = 0
fp = 0
fn = 0

for id in all_test_fig_id:
    if id in gt:
        p = True
    else: p = False
    if id in pred_dict and pred_dict[id] >= b:
        t = True
    else: t = False
    
    if t and p: tp += 1
    if t and not p: fp += 1
    if not t and p: fn += 1

prec = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * prec * rec / (prec + rec)

tp, fp, fn, prec, rec, f1